<a href="https://colab.research.google.com/github/AyushSrivastava27/Aircraft-Engine-Failure-Prediction/blob/main/Aircraft_Engine_Failure_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ayush7232","key":"20952f7e56c6325521d9600d800b9a72"}'}

In [3]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d 'wolfmedal/aero-engine-defect-new'

 99% 141M/142M [00:09<00:00, 18.4MB/s]
100% 142M/142M [00:09<00:00, 15.9MB/s]


In [5]:
!unzip aero-engine-defect-new.zip -d aero-engine-defect-new

Archive:  aero-engine-defect-new.zip
  inflating: aero-engine-defect-new/Aero-engine_defect-detect_new/images/train/1.jpg  
  inflating: aero-engine-defect-new/Aero-engine_defect-detect_new/images/train/10.jpg  
  inflating: aero-engine-defect-new/Aero-engine_defect-detect_new/images/train/10_1.jpg  
  inflating: aero-engine-defect-new/Aero-engine_defect-detect_new/images/train/10_flip.jpg  
  inflating: aero-engine-defect-new/Aero-engine_defect-detect_new/images/train/10_flip0.jpg  
  inflating: aero-engine-defect-new/Aero-engine_defect-detect_new/images/train/10_flip1.jpg  
  inflating: aero-engine-defect-new/Aero-engine_defect-detect_new/images/train/10_rot1.jpg  
  inflating: aero-engine-defect-new/Aero-engine_defect-detect_new/images/train/10_rot2.jpg  
  inflating: aero-engine-defect-new/Aero-engine_defect-detect_new/images/train/10_rot3.jpg  
  inflating: aero-engine-defect-new/Aero-engine_defect-detect_new/images/train/11.jpg  
  inflating: aero-engine-defect-new/Aero-engine_de

In [6]:
!pip install --quiet img2vec_pytorch
print('pip install img2vec complete.')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 882.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.2 MB/s eta 0:00:00
pip install img2ve

In [8]:
pip install arrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.1 MB/s eta 0:00:00


In [9]:
import base64
import pandas as pd
from arrow import now
from glob import glob
from img2vec_pytorch import Img2Vec
from io import BytesIO
from os.path import basename
from PIL import Image
from plotly import express

# we're going to use the updated dataset
GLOB = '/content/aero-engine-defect-new/Aero-engine_defect-detect_new/images'
SIZE = 512
STOP = 500
THUMBNAIL_SIZE = (100, 100)


def embed(model, filename: str):
    with Image.open(fp=filename, mode='r') as image:
        return model.get_vec(image, tensor=True).numpy().reshape(SIZE,)


def flatten(arg):
    return [x for xs in arg for x in xs]

def png(filename: str) -> str:
    with Image.open(fp=filename, mode='r') as image:
        buffer = BytesIO()
        # our images are pretty big; let's shrink the hover images to thumbnail size
        image.resize(size=THUMBNAIL_SIZE).save(buffer, format='png')
        return 'data:image/png;base64,' + base64.b64encode(buffer.getvalue()).decode()

def get_picture_from_glob(arg: str, tag: str, stop: int) -> list:
    time_get = now()
    result = [pd.Series(data=[tag, basename(input_file), embed(model=model, filename=input_file), png(filename=input_file), ],
                        index=['tag', 'name', 'value', 'image'])
        for index, input_file in enumerate(glob(pathname=arg)) if index < stop]
    print('encoded {} data {} rows in {}'.format(tag, len(result), now() - time_get))
    return result

time_start = now()
model = Img2Vec(cuda=False, model='resnet-18')
data_dict = {basename(folder) : folder + '/*.jpg' for folder in glob(GLOB + '/*')}
df = pd.DataFrame(data=flatten(arg=[get_picture_from_glob(arg=value, tag=key, stop=STOP) for key, value in data_dict.items()]))
# we need to add a key so we can join the labels
df['label name'] = df['name'].apply(func=lambda x: x.replace('.jpg', '.txt'))
print('done in {}'.format(now() - time_start))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 168MB/s]


encoded val data 73 rows in 0:00:16.638281
encoded train data 257 rows in 0:00:58.271937
done in 0:01:15.595162


In [12]:
root = '/content/aero-engine-defect-new/Aero-engine_defect-detect_new/labels/'
dfs = []
for subfolder in ['val', 'train']:
    for input_file in glob(pathname=root + subfolder + '/*.txt'):
        current_df = pd.read_csv(filepath_or_buffer=input_file, sep=' ', names=['label', 'w0', 'w1', 'w2', 'w3'])
        current_df['tag'] = [subfolder] * len(current_df)
        current_df['name'] = basename(input_file)
        dfs.append(current_df)
labels_df = pd.concat(objs=dfs, axis=0)
labels_df.head()

all_df = df.merge(right=labels_df, left_on='label name', right_on='name', how='left')

In [13]:
all_df['class'] = all_df['label'].map({0: 'scratch', 1: 'dot', 2: 'crease', 3: 'damage'})

In [14]:
all_df.head()

,tag_x,name_x,value,image,label name,label,w0,w1,w2,w3,tag_y,name_y,class
0,val,19_2.jpg,"[0.023226224, 1.4867519, 1.0439914, 1.5182507,...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",19_2.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,val,1.jpg,"[1.0555197, 0.3864376, 0.02806777, 4.0779214, ...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",1.txt,0,0.544118,0.458802,0.500613,0.682074,val,1.txt,scratch
2,val,1.jpg,"[1.0555197, 0.3864376, 0.02806777, 4.0779214, ...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",1.txt,0,0.544118,0.458802,0.500613,0.682074,train,1.txt,scratch
3,val,5.jpg,"[0.87150794, 0.6789731, 0.5085094, 1.675594, 0...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",5.txt,0,0.849877,0.152934,0.287990,0.218248,val,5.txt,scratch
4,val,5.jpg,"[0.87150794, 0.6789731, 0.5085094, 1.675594, 0...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",5.txt,0,0.736060,0.533762,0.461091,0.432476,val,5.txt,scratch


In [15]:
val_df = all_df[(all_df['tag_x'] == 'val') & (all_df['tag_y'] == 'val')]
train_df = all_df[(all_df['tag_x'] == 'train') & (all_df['tag_y'] == 'train')]
train_df.head()

,tag_x,name_x,value,image,label name,label,w0,w1,w2,w3,tag_y,name_y,class
163,train,2023-04-06 19-34-32-225.jpg,"[0.86489946, 0.15046766, 0.37534967, 1.4434444...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",2023-04-06 19-34-32-225.txt,1,0.284124,0.678648,0.055316,0.054637,train,2023-04-06 19-34-32-225.txt,dot
164,train,2023-04-06 19-34-32-225.jpg,"[0.86489946, 0.15046766, 0.37534967, 1.4434444...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",2023-04-06 19-34-32-225.txt,1,0.366020,0.671459,0.043822,0.044572,train,2023-04-06 19-34-32-225.txt,dot
165,train,2023-04-06 19-36-30-341.jpg,"[2.2710912, 0.006975433, 1.1958213, 2.5281773,...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",2023-04-06 19-36-30-341.txt,0,0.556034,0.544213,0.086207,0.071891,train,2023-04-06 19-36-30-341.txt,scratch
166,train,2023-04-06 19-36-30-341.jpg,"[2.2710912, 0.006975433, 1.1958213, 2.5281773,...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",2023-04-06 19-36-30-341.txt,0,0.470187,0.506830,0.075431,0.061826,train,2023-04-06 19-36-30-341.txt,scratch
167,train,22_flip.jpg,"[0.17406493, 0.41171205, 0.010119853, 2.730492...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",22_flip.txt,3,0.502604,0.765072,0.293811,0.457797,train,22_flip.txt,damage


In [16]:
express.histogram(data_frame=train_df, x='class')

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

forest = RandomForestClassifier(random_state=2024, n_estimators=20)
forest.fit(X=train_df['value'].apply(pd.Series), y=train_df['class'])

print('accuracy: {:5.4f}'.format(accuracy_score(y_true=val_df['class'], y_pred=forest.predict(X=val_df['value'].apply(pd.Series)))))

accuracy: 0.8421


In [24]:
from sklearn.metrics import classification_report
print(classification_report(y_true=val_df['class'], y_pred=forest.predict(X=val_df['value'].apply(pd.Series))))

              precision    recall  f1-score   support

      crease       0.62      1.00      0.77         5
      damage       0.79      0.79      0.79        19
         dot       0.67      1.00      0.80         2
     scratch       0.91      0.84      0.87        50

    accuracy                           0.84        76
   macro avg       0.75      0.91      0.81        76
weighted avg       0.86      0.84      0.84        76

